In [2]:
using DataFrames, WooldridgeDatasets, GLM, Distributions

# Lista 04

## Questão 01
$P_{H0}(|T| > |t_{obs}|) = 2P_{H0}(T < -|t_{obs}|) = 0,086 \implies P_{H0}(T < -|t_{obs}|) = \frac{0,086}{2} = 0,043$

## Questão 03

In [3]:
rdchem = DataFrame(wooldridge("rdchem"))
first(rdchem, 5)

Row,rd,sales,profits,rdintens,profmarg,salessq,lsales,lrd
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,430.6,4570.2,186.9,9.42191,4.08954,2.08867e7,8.42731,6.06518
2,59.0,2830.0,467.0,2.08481,16.5018,8.0089e6,7.94803,4.07754
3,23.5,596.8,107.4,3.93767,17.996,3.5617e5,6.39158,3.157
4,3.5,133.6,-4.3,2.61976,-3.21856,17849.0,4.89485,1.25276
5,1.7,42.0,8.0,4.04762,19.0476,1764.0,3.73767,0.530628


In [4]:
modelo1 = lm(@formula(rdintens ~ log(sales) + profmarg), rdchem)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

rdintens ~ 1 + :(log(sales)) + profmarg

Coefficients:
─────────────────────────────────────────────────────────────────────────
                 Coef.  Std. Error     t  Pr(>|t|)   Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────────────────
(Intercept)  0.472254    1.67606    0.28    0.7801  -2.95567     3.90017
log(sales)   0.321348    0.215569   1.49    0.1468  -0.119539    0.762236
profmarg     0.0500367   0.0457768  1.09    0.2834  -0.0435874   0.143661
─────────────────────────────────────────────────────────────────────────

### Item a

$\Delta y = (\beta_1 / 100)\% \Delta x$

Quando `sales` aumenta em 1%, é esperado que `rdintens` aumente 0,003

## Questão 04

In [5]:
gpa1 = DataFrame(wooldridge("gpa1"))
first(gpa1, 5)

Row,age,soph,junior,senior,senior5,male,campus,business,engineer,colGPA,hsGPA,ACT,job19,job20,drive,bike,walk,voluntr,PC,greek,car,siblings,bgfriend,clubs,skipped,alcohol,gradMI,fathcoll,mothcoll
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Int64,Int64,Int64
1,21,0,0,1,0,0,0,1,0,3.0,3.0,21,0,1,1,0,0,0,0,0,1,1,0,0,2.0,1.0,1,0,0
2,21,0,0,1,0,0,0,1,0,3.4,3.2,24,0,1,1,0,0,0,0,0,1,0,1,1,0.0,1.0,1,1,1
3,20,0,1,0,0,0,0,1,0,3.0,3.6,26,1,0,0,0,1,0,0,0,1,1,0,1,0.0,1.0,1,1,1
4,19,1,0,0,0,1,1,1,0,3.5,3.5,27,1,0,0,0,1,0,0,0,0,1,0,0,0.0,0.0,0,0,0
5,20,0,1,0,0,0,0,1,0,3.6,3.9,28,0,1,0,1,0,0,0,0,1,1,1,0,0.0,1.5,1,1,0


In [6]:
modelo2 = lm(@formula(colGPA ~ skipped + hsGPA + ACT), gpa1)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

colGPA ~ 1 + skipped + hsGPA + ACT

Coefficients:
─────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)    Lower 95%   Upper 95%
─────────────────────────────────────────────────────────────────────────────
(Intercept)   1.38955     0.331554    4.19    <1e-04   0.73393      2.04518
skipped      -0.0831131   0.0259985  -3.20    0.0017  -0.134523    -0.0317028
hsGPA         0.411816    0.0936742   4.40    <1e-04   0.226582     0.59705
ACT           0.0147202   0.0105649   1.39    0.1658  -0.00617107   0.0356115
─────────────────────────────────────────────────────────────────────────────

### Item a

$P(-t_{0,025; n-k-1} < \frac{\hat{\beta_{hsGPA}} - \beta_{hsGPA}}{\sqrt{V(\hat{\beta_{hsGPA}} | X)}} < t_{0,025; n-k-1}) = 0,95$

In [7]:
IC95_inf = coef(modelo2)[3] + stderror(modelo2)[3] * quantile(TDist(nrow(gpa1) - 4), 0.025)
IC95_sup = coef(modelo2)[3] - stderror(modelo2)[3] * quantile(TDist(nrow(gpa1) - 4), 0.025)

IC95_inf, IC95_sup

(0.22658185110578052, 0.5970504923693317)

### Item b

$H_0: \beta_{hsGPA} = 0,4 \ vs. \ H_1: \beta_{hsGPA} \neq 0,4 \\$ 
$pvalor = P_{H0}(|T| > |t_{obs}|)$

In [15]:
tobs = (coef(modelo2)[3] - 0.4) / stderror(modelo2)[3]
pvalor = 2 * cdf(TDist(nrow(gpa1)-4), -tobs)


0.8998051414602578

In [18]:
tobs = (coef(modelo2)[3] - 1) / stderror(modelo2)[3]
pvalor = 2 * cdf(TDist(nrow(gpa1)-4), tobs)

4.204251632295657e-9

### Item d
$H_0: \beta_{skipped} = \beta_{ACT} = 0 \ vs. \ H_1: \text{pelo menos 1 é diferente de 0}$ 

In [40]:
modelor = lm(@formula(colGPA ~ hsGPA), gpa1)
SQRi = sum(residuals(modelo2) .^ 2)
SQRr = sum(residuals(modelor) .^ 2)

Fobs = ((SQRr-SQRi)/SQRi) * ((nrow(gpa1)-4)/2)
pvalor = 1 - cdf(FDist(2, nrow(gpa1)-4), Fobs)

0.004957052819108521

## Questão 05

In [42]:
wage2 = DataFrame(wooldridge("wage2"))
first(wage2, 5)

Row,wage,hours,IQ,KWW,educ,exper,tenure,age,married,black,south,urban,sibs,brthord,meduc,feduc,lwage
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64?,Int64?,Int64?,Float64
1,769,40,93,35,12,11,2,31,1,0,0,1,1,2,8,8,6.64509
2,808,50,119,41,18,11,16,37,1,0,0,1,1,missing,14,14,6.69456
3,825,40,108,46,14,11,9,33,1,0,0,1,1,2,14,14,6.71538
4,650,40,96,32,12,13,7,32,1,0,0,1,4,3,12,12,6.47697
5,562,40,74,27,11,14,5,34,1,0,0,1,10,6,6,11,6.3315


In [43]:
modelo3 = lm(@formula(log(wage) ~ educ + exper + tenure), wage2)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

:(log(wage)) ~ 1 + educ + exper + tenure

Coefficients:
──────────────────────────────────────────────────────────────────────────
                 Coef.  Std. Error      t  Pr(>|t|)   Lower 95%  Upper 95%
──────────────────────────────────────────────────────────────────────────
(Intercept)  5.4967     0.110528    49.73    <1e-99  5.27978     5.71361
educ         0.0748638  0.00651245  11.50    <1e-27  0.062083    0.0876446
exper        0.0153285  0.00336957   4.55    <1e-05  0.00871564  0.0219413
tenure       0.0133748  0.00258719   5.17    <1e-06  0.00829739  0.0184522
──────────────────────────────────────────────────────────────────────────

### Item a


In [47]:
modelor = lm(@formula(log(wage) ~ 1), wage2);
ftest(modelor.model, modelo3.model)


F-test: 2 models fitted on 935 observations
───────────────────────────────────────────────────────────────────
     DOF  ΔDOF       SSR      ΔSSR      R²     ΔR²       F*   p(>F)
───────────────────────────────────────────────────────────────────
[1]    2        165.6563            0.0000                         
[2]    5     3  139.9610  -25.6953  0.1551  0.1551  56.9739  <1e-33
───────────────────────────────────────────────────────────────────

### Item b
$\beta_2 = \beta_3 \implies \beta_2 - \beta_3 = 0$

In [62]:
wage2.newvar = wage2.exper - wage2.tenure

modelor = lm(@formula(log(wage) ~ educ + newvar), wage2)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

:(log(wage)) ~ 1 + educ + newvar

Coefficients:
───────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error      t  Pr(>|t|)    Lower 95%    Upper 95%
───────────────────────────────────────────────────────────────────────────────
(Intercept)   6.03388     0.0890591   67.75    <1e-99   5.8591      6.20866
educ          0.0565914   0.00626597   9.03    <1e-18   0.0442944   0.0688885
newvar       -0.00394292  0.00235792  -1.67    0.0948  -0.00857037  0.000684522
───────────────────────────────────────────────────────────────────────────────

### Item c

In [51]:
modelor = lm(@formula(log(wage) ~ tenure), wage2)

SQRi = sum(residuals(modelo3) .^ 2)
SQRr = sum(residuals(modelor) .^ 2)

Fobs = ((SQRr-SQRi)/SQRi) * ((nrow(wage2)-4)/2)
pvalor = 1 - cdf(FDist(2, nrow(wage2)-4), Fobs)

0.0

## Questão 06

In [53]:
htv = DataFrame(wooldridge("htv"))
first(htv, 5)

Row,wage,abil,educ,ne,nc,west,south,exper,motheduc,fatheduc,brkhme14,sibs,urban,ne18,nc18,south18,west18,urban18,tuit17,tuit18,lwage,expersq,ctuit
,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Float64
1,12.0192,5.02774,15,0,0,1,0,9,12,12,0,1,1,1,0,0,0,1,7.58291,7.26024,2.48651,81,-0.322671
2,8.91266,2.03717,13,1,0,0,0,8,12,10,1,4,1,1,0,0,0,1,8.59514,9.49954,2.18747,64,0.904392
3,15.5143,2.4759,15,1,0,0,0,11,12,16,0,2,1,1,0,0,0,1,7.31135,7.31135,2.74176,121,0.0
4,13.3333,3.60924,15,1,0,0,0,6,12,12,0,1,1,1,0,0,0,1,9.49954,10.1621,2.59027,36,0.662534
5,11.0701,2.63655,13,1,0,0,0,15,12,15,1,2,1,1,0,0,0,1,7.31135,7.31135,2.40425,225,0.0


In [54]:
modelo4 = lm(@formula(educ ~ motheduc + fatheduc + abil + abil^2), htv)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

educ ~ 1 + motheduc + fatheduc + abil + :(abil ^ 2)

Coefficients:
────────────────────────────────────────────────────────────────────────
                Coef.  Std. Error      t  Pr(>|t|)  Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────────
(Intercept)  8.24023    0.28741    28.67    <1e-99  7.67636    8.8041
motheduc     0.190126   0.0280957   6.77    <1e-10  0.135005   0.245247
fatheduc     0.108939   0.0196014   5.56    <1e-07  0.0704827  0.147395
abil         0.401462   0.0302875  13.26    <1e-36  0.342041   0.460884
abil ^ 2     0.050599   0.0083039   6.09    <1e-08  0.0343076  0.0668905
────────────────────────────────────────────────────────────────────────

### Item a

In [58]:
htv.newvar = htv.motheduc - htv.fatheduc

modelor = lm(@formula(educ ~ newvar + abil + abil^2), htv)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

educ ~ 1 + newvar + abil + :(abil ^ 2)

Coefficients:
─────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error       t  Pr(>|t|)   Lower 95%   Upper 95%
─────────────────────────────────────────────────────────────────────────────
(Intercept)  11.6642     0.0788618   147.91    <1e-99  11.5095     11.819
newvar       -0.062451   0.0203867    -3.06    0.0022  -0.102448   -0.0224544
abil          0.526216   0.0302571    17.39    <1e-59   0.466855    0.585578
abil ^ 2      0.0513976  0.00880056    5.84    <1e-08   0.0341318   0.0686635
─────────────────────────────────────────────────────────────────────────────

### Item c

In [60]:
modelor = lm(@formula(educ ~ motheduc + fatheduc), htv)

SQRi = sum(residuals(modelo4) .^ 2)
SQRr = sum(residuals(modelor) .^ 2)

Fobs = ((SQRr-SQRi)/SQRi) * ((nrow(htv)-5)/2)
pvalor = 1 - cdf(FDist(2, nrow(htv)-5), Fobs)

0.0